In [1]:
! pip install -U spacy
! python -m spacy download pt_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-21 16:05:23.633150: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 568.2 MB 14 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [2]:
#import all necessary libraries
#supervised and unsupervised (not part of proposal) classification. FastText is used for both. For the flat, just using traditional ML models. 

import pandas as pd
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer

# models
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

# testing
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

# pipeline 
from sklearn.pipeline import Pipeline

# metrics
from sklearn.metrics import classification_report

In [3]:
# pre-processing
import re
import string 

import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
nltk.download('punkt')
punctuation = set(string.punctuation)

# lemma in portuguese
import spacy
import pt_core_news_lg
nlp = spacy.load('pt_core_news_lg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
#get hierarchical_data

df = pd.read_csv('https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv')
print(df.shape)
df.head(2)

(5668, 80)


,text,Hate.speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Thin.women,Arabic,East.europeans,Africans,South.Americans,Brazilians,Migrants,Homossexuals,Thin.people,Ageing
0,"""não come mel, morde marimbondo""",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"não tem pinto, tem orgulho !",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#pre-processing function
stopwords_manual = ['http?', 'mais', 'is?o', 'es[st]?*', 'quan[dt]?', ' ', '\n', '...', 'de o', 'em o', 'rt', 'ter', 'pra', 'a o', 'q', '  ', '..', 'por 0', 'fazer', 'dizer', 'vc']

def preprocessing(text):

    text = re.sub('@[\w]+','',text) #remove usernames
    text = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', text) #remove links
    text = [w.lemma_ for w in nlp(text)] #This lemma also performs tokenization
    text = [word for word in text if word not in punctuation]
    text = [w.lower() for w in text]
    text = [word for word in text if word not in stopwords and word not in stopwords_manual]
    text = ' '.join([str(word) for word in text])

    return text

In [6]:
#apply pre=processing

df['text'] = df.text.apply(preprocessing)

In [7]:
# selecting X and y variables
X = df.text
y = df["Hate.speech"]

In [8]:
# 10-fold crossvalidation (Chollet, 2017), combined with holdout validation

def model_evaluate(X, y, model, model_name, n_splits:int=10):
  
    # kfold with 10 splits split
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # set a pipeline
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('classifier', model)
    ])

    # metrics
    scoring = ['accuracy', 'f1', 'f1_micro', 'f1_macro', 'precision', 'recall']

    # cross validation
    scores = cross_validate(pipeline, X, y, scoring=scoring, cv=kf)

    # format the results into a dataframe
    results = pd.DataFrame(scores)
    results['model_name'] = model_name
    # save it into the results directory
#     results.to_csv(f'Data/{model_name}.csv')
    
    return results
    # return None

In [9]:
#the models we are using for binary classification
models = {
    'ComplementNB':ComplementNB(),
    'LogisticRegression':LogisticRegression(),
    'ForestClassifier':RandomForestClassifier(),
    'xgboost': xgb.XGBClassifier()
}

In [10]:
#results for binary classification on all categories
results = []
results_list = []
for k, v in models.items():
    results = model_evaluate(X, y, v, k)
    results_list.append(results)

In [11]:
total_results = pd.concat(results_list)

In [12]:
round(total_results.groupby('model_name').mean(), 3)

,fit_time,score_time,test_accuracy,test_f1,test_f1_micro,test_f1_macro,test_precision,test_recall
model_name,,,,,,,,
ComplementNB,0.080,0.014,0.833,0.598,0.833,0.747,0.627,0.573
ForestClassifier,3.275,0.091,0.878,0.659,0.878,0.792,0.835,0.545
LogisticRegression,0.209,0.013,0.860,0.570,0.860,0.744,0.853,0.429
xgboost,1.529,0.017,0.865,0.590,0.865,0.754,0.856,0.450


In [13]:
def make_models(y, X=X, models=models):
    results_list = []
    for k, v in models.items():
        results = model_evaluate(X, y, v, k)
        results_list.append(results)
    total_results = pd.concat(results_list)
    return total_results.groupby('model_name').mean()

In [14]:
results_list = make_models(y)

In [15]:
#results for binary classification on each category 

results = []
for col in df.columns[1:]:
    print(col)
    r = make_models(df[str(col)])
    r['var'] = col
    results.append(r)

Hate.speech
Sexism
Body
Racism


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

Ideology


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

Homophobia
Origin


...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

Homossexuals
Thin.people


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

Ageing


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to 

In [16]:
flat_per_categ = pd.concat(results)
flat_per_categ.head()

,fit_time,score_time,test_accuracy,test_f1,test_f1_micro,test_f1_macro,test_precision,test_recall,var
model_name,,,,,,,,,
ComplementNB,0.073198,0.011796,0.833279,0.598412,0.833279,0.746587,0.627416,0.573264,Hate.speech
ForestClassifier,3.156815,0.091136,0.876850,0.653828,0.876850,0.789462,0.835682,0.537432,Hate.speech
LogisticRegression,0.210625,0.011770,0.860445,0.570453,0.860445,0.743567,0.852571,0.429155,Hate.speech
xgboost,1.564982,0.017938,0.864679,0.589703,0.864679,0.754336,0.856496,0.450347,Hate.speech
ComplementNB,0.073569,0.012068,0.885500,0.488370,0.885500,0.711937,0.522531,0.461282,Sexism


In [17]:
flat_per_categ.groupby('var').mean()

,fit_time,score_time,test_accuracy,test_f1,test_f1_micro,test_f1_macro,test_precision,test_recall
var,,,,,,,,
Aborting.women,0.565734,0.021431,0.991973,0.000000,0.991973,0.685464,0.000000,0.000000
Africans,0.497847,0.019451,0.992061,0.002941,0.992061,0.711956,0.001563,0.025000
Ageing,0.496338,0.019334,0.991796,0.000000,0.991796,0.685418,0.000000,0.000000
Agnostic,0.326309,0.018646,0.993340,0.000000,0.993340,0.848309,0.000000,0.000000
Angolans,0.497515,0.019293,0.992061,0.002941,0.992061,0.711956,0.001563,0.025000
...,...,...,...,...,...,...,...,...
Vegetarians,0.443586,0.020547,0.992369,0.000000,0.992369,0.798062,0.000000,0.000000
Venezuelans,0.347674,0.018874,0.993296,0.000000,0.993296,0.848298,0.000000,0.000000
White.people,0.478663,0.024073,0.992282,0.000000,0.992282,0.798040,0.000000,0.000000
